##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# CartPole con método Actor-Crítico


<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/tutorials/reinforcement_learning/actor_critic">     <img src="https://www.tensorflow.org/images/tf_logo_32px.png">     Ver en TensorFlow.org</a> </td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/es-419/tutorials/reinforcement_learning/actor_critic.ipynb">     <img src="https://www.tensorflow.org/images/colab_logo_32px.png">     Ejecutar en Google Colab</a> </td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/es-419/tutorials/reinforcement_learning/actor_critic.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fuente en GitHub</a> </td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/es-419/tutorials/reinforcement_learning/actor_critic.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Descargar notebook</a></td>
</table>

En este tutorial se demuestra cómo implementar un método [Actor-Critic](https://papers.nips.cc/paper/1786-actor-critic-algorithms.pdf) (actor-crítico) con TensorFlow para entrenar a un agente en el entorno [Open AI Gym](https://www.gymlibrary.dev/) [`CartPole-v0`](https://www.gymlibrary.dev/environments/classic_control/cart_pole/) (carro con mástil). Se supone que el lector está, en cierto modo, familiarizado con los [métodos de gradientes de política](https://papers.nips.cc/paper/1713-policy-gradient-methods-for-reinforcement-learning-with-function-approximation.pdf) del [aprendizaje por refuerzo (profundo)](https://en.wikipedia.org/wiki/Deep_reinforcement_learning).


**Los métodos actor-crítico**

Los métodos actor-crítico son [métodos de aprendizaje por diferencia temporal (TD)](https://en.wikipedia.org/wiki/Temporal_difference_learning) que representan la función política independiente de la función valor.

Una función de política (o política) devuelve una distribución de probabilidad sobre acciones que el agente puede tomar basándose en un estado dado. La función de valor determina el retorno esperado para un agente que se inicia en un estado dado y actúa según una política particular de allí en adelante.

En el método actor-crítico, nos referimos a la política como el *actor* que propone un conjunto de acciones posibles en un estado dado y la función de valor estimado es el *crítico*, que es quien evalúa las acciones realizadas por el *actor* basándose en la política dada.

En este tutorial, tanto el *actor* como el *crítico* estarán representados por una red neuronal con dos salidas.


**`CartPole-v0`**

En el entorno [`CartPole-v0` ](https://www.gymlibrary.dev/environments/classic_control/cart_pole/)hay un mástil en posición vertical sobre un carro que se mueve a lo largo de un riel sin fricciones. El mástil empieza derecho y el objetivo del agente es evitar que se caiga aplicando una fuerza de `-1` o `+1` al carro. Por cada vez que el mástil permanece en posición vertical, se otorga una recompensa de `+1`. Un episodio termina cuando: 1) el mástil se inclina más de 15 grados (desde la posición vertical); o 2) el carro se mueve más de 2.4 unidades del centro.

<center>
  <pre data-md-type="custom_pre">&lt;figure&gt;
    &lt;image src="https://tensorflow.org/tutorials/reinforcement_learning/images/cartpole-v0.gif"&gt;
    &lt;figcaption&gt;
      Trained actor-critic model in Cartpole-v0 environment
    &lt;/figcaption&gt;
  &lt;/figure&gt;</pre>
</center>


El problema se considera "resuelto" cuando el promedio de recompensas totales para un episodio llega a 195 sobre 100 pruebas consecutivas.

## Preparar

Importe los paquetes necesarios y configure los ajustes globales.


In [ ]:
!pip install gym[classic_control]
!pip install pyglet

In [ ]:
%%bash
# Install additional packages for visualization
sudo apt-get install -y python-opengl > /dev/null 2>&1
pip install git+https://github.com/tensorflow/docs > /dev/null 2>&1

In [ ]:
import collections
import gym
import numpy as np
import statistics
import tensorflow as tf
import tqdm

from matplotlib import pyplot as plt
from tensorflow.keras import layers
from typing import Any, List, Sequence, Tuple


# Create the environment
env = gym.make("CartPole-v1")

# Set seed for experiment reproducibility
seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)

# Small epsilon value for stabilizing division operations
eps = np.finfo(np.float32).eps.item()

## El modelo

El *actor* y el *crítico* se modelarán usando una red neuronal que genere las probabilidades de acción y el respectivo valor crítico. En este tutorial se usa el modelo de subclases para definir el modelo.

Durante el pase hacia adelante, el modelo tomará el estado de la entrada y saldrán las dos probabilidades de acción y el valor crítico $V$, que modela la [función de valor](https://spinningup.openai.com/en/latest/spinningup/rl_intro.html#value-functions) dependiente del estado. El objetivo es entrenar un modelo que elija acciones basándose en una política $\pi$ que maximice el [retorno](https://spinningup.openai.com/en/latest/spinningup/rl_intro.html#reward-and-return) esperado.

Para `CartPole-v0`, hay cuatro valores que representan al estado: la posición del carro, la velocidad del carro, el ángulo del mástil y la velocidad respectiva. El agente puede realizar dos acciones para empujar el carro hacia la izquierda (`0`) y la derecha (`1`), respectivamente.

Para más información, consulte la [página de documentación sobre Cart Pole de Gym](https://www.gymlibrary.dev/environments/classic_control/cart_pole/) y los [*Neuronlike adaptive elements that can solve difficult learning control problems*](http://www.derongliu.org/adp/adp-cdrom/Barto1983.pdf) (Elementos adaptativos de tipo neuronal que pueden resolver problemas difíciles de control de aprendizaje) por Barto, Sutton y Anderson (1983).


In [ ]:
class ActorCritic(tf.keras.Model):
  """Combined actor-critic network."""

  def __init__(
      self, 
      num_actions: int, 
      num_hidden_units: int):
    """Initialize."""
    super().__init__()

    self.common = layers.Dense(num_hidden_units, activation="relu")
    self.actor = layers.Dense(num_actions)
    self.critic = layers.Dense(1)

  def call(self, inputs: tf.Tensor) -> Tuple[tf.Tensor, tf.Tensor]:
    x = self.common(inputs)
    return self.actor(x), self.critic(x)

In [ ]:
num_actions = env.action_space.n  # 2
num_hidden_units = 128

model = ActorCritic(num_actions, num_hidden_units)

## Entrenamiento del agente

Para entrenar el agente deberá seguir los pasos que se encuentran a continuación:

1. Ejecutar el agente en el entorno para recopilar datos de entrenamiento por episodio.
2. Calcular el retorno esperado a cada paso de tiempo.
3. Calcular la pérdida para el modelo actor-crítico combinado.
4. Calcular gradientes y actualizar los parámetros de red.
5. Repetir los pasos 1 a 4 hasta alcanzar lo establecido según el criterio de éxito o la cantidad máxima de episodios.


### 1. Recopilación de datos de entrenamiento

Tal como en el aprendizaje supervisado, para entrenar el modelo actor-crítico, deberá entrenar los datos. Pero, para recopilar tales datos, el modelo debería "ejecutarse" en el entorno.

Los datos de entrenamiento se recopilan para cada episodio. Después, a cada paso de tiempo se ejecutará el pase hacia adelante del modelo en el estado del entorno a fin de generar probabilidades de acción y el valor crítico, según la política actual, parametrizado por los pesos del modelo.

La siguiente acción se muestreará a partir de probabilidades de acciones generadas por el modelo. Después se aplicarían al entorno, causando el siguiente estado y la recompensa.

Este proceso se implementa en la función `run_episode` que usa operaciones de TensorFlow para su posterior compilación en un gráfico de TensorFlow, para lograr un entrenamiento más rápido. Tenga en cuenta que los `tf.TensorArray` se usaron para admitir la iteración Tensor en arreglos de longitud variable.

In [ ]:
# Wrap Gym's `env.step` call as an operation in a TensorFlow function.
# This would allow it to be included in a callable TensorFlow graph.

def env_step(action: np.ndarray) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
  """Returns state, reward and done flag given an action."""

  state, reward, done, truncated, info = env.step(action)
  return (state.astype(np.float32), 
          np.array(reward, np.int32), 
          np.array(done, np.int32))


def tf_env_step(action: tf.Tensor) -> List[tf.Tensor]:
  return tf.numpy_function(env_step, [action], 
                           [tf.float32, tf.int32, tf.int32])

In [ ]:
def run_episode(
    initial_state: tf.Tensor,  
    model: tf.keras.Model, 
    max_steps: int) -> Tuple[tf.Tensor, tf.Tensor, tf.Tensor]:
  """Runs a single episode to collect training data."""

  action_probs = tf.TensorArray(dtype=tf.float32, size=0, dynamic_size=True)
  values = tf.TensorArray(dtype=tf.float32, size=0, dynamic_size=True)
  rewards = tf.TensorArray(dtype=tf.int32, size=0, dynamic_size=True)

  initial_state_shape = initial_state.shape
  state = initial_state

  for t in tf.range(max_steps):
    # Convert state into a batched tensor (batch size = 1)
    state = tf.expand_dims(state, 0)
  
    # Run the model and to get action probabilities and critic value
    action_logits_t, value = model(state)
  
    # Sample next action from the action probability distribution
    action = tf.random.categorical(action_logits_t, 1)[0, 0]
    action_probs_t = tf.nn.softmax(action_logits_t)

    # Store critic values
    values = values.write(t, tf.squeeze(value))

    # Store log probability of the action chosen
    action_probs = action_probs.write(t, action_probs_t[0, action])
  
    # Apply action to the environment to get next state and reward
    state, reward, done = tf_env_step(action)
    state.set_shape(initial_state_shape)
  
    # Store reward
    rewards = rewards.write(t, reward)

    if tf.cast(done, tf.bool):
      break

  action_probs = action_probs.stack()
  values = values.stack()
  rewards = rewards.stack()
  
  return action_probs, values, rewards

### 2. Cálculo de los retornos esperados

La secuencia de recompensas para cada paso de tiempo recopilado $t$, ${r_{t}}^{T}{em0}{t=1}$ durante un episodio se convierte en una secuencia de retornos esperados ${G{/em0}{t}}^{T}_{t=1}$ en la que la suma de recompensas se toma del paso de tiempo actual $t$ a $T$ y cada recompensa se multiplica por un factor $\gamma$ de descuento que decae exponencialmente:

$$G_{t} = \sum^{T}_{t'=t} \gamma^{t'-t}r_{t'}$$

Dado que $\gamma\in(0,1)$, se les asigna menos peso a las recompensas más alejadas del paso de tiempo actual.

Intuitivamente, el retorno esperado simplemente implica que las recompensas de ahora son mejores que las de más adelante. En un sentido matemático, se trata de garantizar que la suma de las recompensas converja.

Para estabilizar el entrenamiento, la secuencia resultante de retornos también se estandariza (es decir, para tener promedio cero y desvío estándar unitario).


In [ ]:
def get_expected_return(
    rewards: tf.Tensor, 
    gamma: float, 
    standardize: bool = True) -> tf.Tensor:
  """Compute expected returns per timestep."""

  n = tf.shape(rewards)[0]
  returns = tf.TensorArray(dtype=tf.float32, size=n)

  # Start from the end of `rewards` and accumulate reward sums
  # into the `returns` array
  rewards = tf.cast(rewards[::-1], dtype=tf.float32)
  discounted_sum = tf.constant(0.0)
  discounted_sum_shape = discounted_sum.shape
  for i in tf.range(n):
    reward = rewards[i]
    discounted_sum = reward + gamma * discounted_sum
    discounted_sum.set_shape(discounted_sum_shape)
    returns = returns.write(i, discounted_sum)
  returns = returns.stack()[::-1]

  if standardize:
    returns = ((returns - tf.math.reduce_mean(returns)) / 
               (tf.math.reduce_std(returns) + eps))

  return returns

### 3. Pérdida en el modelo actor-crítico

Dado que está usando un modelo híbrido actor-crítico, la función de pérdida elegida es una combinación de las pérdidas de actor y de crítico para el entrenamiento, tal como se muestra a continuación:

$$L = L_{actor} + L_{critic}$$

#### La pérdida de actor

La pérdida de actor se basa en [gradientes de políticas con el crítico como línea de base dependiente del estado](https://www.youtube.com/watch?v=EKqxumCuAAY&t=62m23s) y se calcula con estimaciones de una sola muestra (por episodio).

$$L_{actor} = -\sum^{T}_{t=1} \log\pi_{\theta}(a_{t} | s_{t})[G(s_{t}, a_{t})  - V^{\pi}_{\theta}(s_{t})]$$

donde:

- $T$: la cantidad de pasos de tiempo por episodio que puede variar por episodio
- $s_{t}$: el estado al paso de tiempo $t$
- $a_{t}$: la acción elegida al paso de tiempo a un $t$ estado dado $s$
- $\pi_{\theta}$: es la política (actor) parametrizada por $\theta$
- $V^{\pi}_{\theta}$: es la función de valor (crítico) también parametrizada por $\theta$
- $G = G_{t}$: el retorno esperado para un estado dado, se empareja con la acción en el paso de tiempo $t$

Se agrega un término negativo a la suma, ya que la idea es maximizar las probabilidades de acciones que produzcan mayores recompensas al minimizar la pérdida combinada.

<br>

##### La ventaja

El término $G - V$ en nuestra formulación $L_{actor}$ se denomina [ventaja](https://spinningup.openai.com/en/latest/spinningup/rl_intro.html#advantage-functions), lo que indica en qué medida a una acción se le da mejor un estado dado sobre una acción aleatoria seleccionada según la política $\pi$ para ese estado.

Si bien es posible excluir una línea de base, hacerlo podría causar una variación grande durante el entrenamiento. Y lo bueno de elegir el crítico $V$ como base es que está entrenado para estar lo más cerca posible de $G$, lo que lleva a una variación inferior.

Además, sin el crítico, el algoritmo intentaría las probabilidades para las acciones realizadas en un estado en particular basándose en el retorno esperado, que puede no hacer una gran diferencia si las probabilidades relativas entre las acciones siguen siendo las mismas.

Por ejemplo, supongamos que dos acciones para un estado dado produjeran el mismo retorno esperado. Sin el crítico, el algoritmo intentaría elevar las probabilidades de esas acciones basándose en el objetivo $J$. Con el crítico, puede resultar que no haya ventaja ($G - V = 0$) y, por lo tanto, no se obtenga ningún beneficio al aumentar las probabilidades de las acciones. Además, el algoritmo establecería los gradientes en cero.

<br>

#### La pérdida de crítico

El entrenamiento de $V$ para que esté lo más cerca posible de $G$ se puede configurar como un problema de regresión con la siguiente función de pérdida:

$$L_{critic} = L_{\delta}(G, V^{\pi}_{\theta})$$

donde $L_{\delta}$ es la [pérdida Huber](https://en.wikipedia.org/wiki/Huber_loss), que es menos sensible a los valores atípicos (outliers) en los datos que la pérdida del error al cuadrado.


In [ ]:
huber_loss = tf.keras.losses.Huber(reduction=tf.keras.losses.Reduction.SUM)

def compute_loss(
    action_probs: tf.Tensor,  
    values: tf.Tensor,  
    returns: tf.Tensor) -> tf.Tensor:
  """Computes the combined Actor-Critic loss."""

  advantage = returns - values

  action_log_probs = tf.math.log(action_probs)
  actor_loss = -tf.math.reduce_sum(action_log_probs * advantage)

  critic_loss = huber_loss(values, returns)

  return actor_loss + critic_loss

### 4. Definición del paso de entrenamiento para actualizar los parámetros

Todos los pasos anteriores se combinan para formar un paso de entrenamiento que se ejecuta en cada episodio. Todos los pasos que llevan a la función de pérdida se ejecutan con el contexto `tf.GradientTape` para permitir la diferenciación automática.

En este tutorial se usa el optimizador Adam para aplicar los gradientes a los parámetros del modelo.

En este paso también se calcula la suma de recompensas no descontadas, `episode_reward`. Este valor se usará más adelante para evaluar si se cumple con el criterio de éxito.

El contexto `tf.function` se aplica a la función `train_step` para que se pueda compilar en un gráfico TensorFlow invocable, que puede aumentar 10 veces la velocidad en el entrenamiento.


In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)


@tf.function
def train_step(
    initial_state: tf.Tensor, 
    model: tf.keras.Model, 
    optimizer: tf.keras.optimizers.Optimizer, 
    gamma: float, 
    max_steps_per_episode: int) -> tf.Tensor:
  """Runs a model training step."""

  with tf.GradientTape() as tape:

    # Run the model for one episode to collect training data
    action_probs, values, rewards = run_episode(
        initial_state, model, max_steps_per_episode) 

    # Calculate the expected returns
    returns = get_expected_return(rewards, gamma)

    # Convert training data to appropriate TF tensor shapes
    action_probs, values, returns = [
        tf.expand_dims(x, 1) for x in [action_probs, values, returns]] 

    # Calculate the loss values to update our network
    loss = compute_loss(action_probs, values, returns)

  # Compute the gradients from the loss
  grads = tape.gradient(loss, model.trainable_variables)

  # Apply the gradients to the model's parameters
  optimizer.apply_gradients(zip(grads, model.trainable_variables))

  episode_reward = tf.math.reduce_sum(rewards)

  return episode_reward

### 5. Ejecución del bucle de entrenamiento

El entrenamiento se lleva a cabo ejecutando el paso de entrenamiento hasta alcanzar lo establecido según el criterio de éxito o hasta alcanzar la cantidad máxima de episodios.

En una cola se guarda un registro en ejecución de recompensas del episodio. Una vez que se llega a las 100 pruebas, la recompensa más antigua se elimina del extremo de la izquierda (extremo de la hilera) de la cola y la nueva se agrega a la cabeza (derecha). También se mantiene una suma de las recompensas en ejecución, para eficiencia en los cálculos.

Dependiendo del tiempo de ejecución, el entrenamiento puede terminar en menos de un minuto.

In [ ]:
%%time

min_episodes_criterion = 100
max_episodes = 10000
max_steps_per_episode = 500

# `CartPole-v1` is considered solved if average reward is >= 475 over 500 
# consecutive trials
reward_threshold = 475
running_reward = 0

# The discount factor for future rewards
gamma = 0.99

# Keep the last episodes reward
episodes_reward: collections.deque = collections.deque(maxlen=min_episodes_criterion)

t = tqdm.trange(max_episodes)
for i in t:
    initial_state, info = env.reset()
    initial_state = tf.constant(initial_state, dtype=tf.float32)
    episode_reward = int(train_step(
        initial_state, model, optimizer, gamma, max_steps_per_episode))
    
    episodes_reward.append(episode_reward)
    running_reward = statistics.mean(episodes_reward)
  

    t.set_postfix(
        episode_reward=episode_reward, running_reward=running_reward)
  
    # Show the average episode reward every 10 episodes
    if i % 10 == 0:
      pass # print(f'Episode {i}: average reward: {avg_reward}')
  
    if running_reward > reward_threshold and i >= min_episodes_criterion:  
        break

print(f'\nSolved at episode {i}: average reward: {running_reward:.2f}!')

## Visualización

Después del entrenamiento, será bueno visualizar cómo se comporta el modelo en el entorno. Lo que puede hacer es ejecutar las celdas que se encuentran a continuación para generar una animación GIF de la ejecución de un episodio del modelo. Tenga en cuenta que debe instalar paquetes adicionales para que Gym renderice las imágenes del entorno correctamente en Colab.

In [ ]:
# Render an episode and save as a GIF file

from IPython import display as ipythondisplay
from PIL import Image

render_env = gym.make("CartPole-v1", render_mode='rgb_array')

def render_episode(env: gym.Env, model: tf.keras.Model, max_steps: int): 
  state, info = env.reset()
  state = tf.constant(state, dtype=tf.float32)
  screen = env.render()
  images = [Image.fromarray(screen)]
 
  for i in range(1, max_steps + 1):
    state = tf.expand_dims(state, 0)
    action_probs, _ = model(state)
    action = np.argmax(np.squeeze(action_probs))

    state, reward, done, truncated, info = env.step(action)
    state = tf.constant(state, dtype=tf.float32)

    # Render screen every 10 steps
    if i % 10 == 0:
      screen = env.render()
      images.append(Image.fromarray(screen))
  
    if done:
      break
  
  return images


# Save GIF image
images = render_episode(render_env, model, max_steps_per_episode)
image_file = 'cartpole-v1.gif'
# loop=0: loop forever, duration=1: play each frame for 1ms
images[0].save(
    image_file, save_all=True, append_images=images[1:], loop=0, duration=1)

In [ ]:
import tensorflow_docs.vis.embed as embed
embed.embed_file(image_file)

## Próximos pasos

En este tutorial demostramos cómo implementar el método actor-crítico en TensorFlow.

Un paso siguiente podría ser el de entrenar un modelo en un entorno diferente en Gym.

Para más información sobre el método actor-crítico y el problema Cartpole-v0, puede consultar los siguientes recursos:

- [El método actor-crítico](https://hal.inria.fr/hal-00840470/document)
- [Exposición sobre actor-crítico (CAL)](https://www.youtube.com/watch?v=EKqxumCuAAY&list=PLkFD6_40KJIwhWJpGazJ9VSj9CFMkb79A&index=7&t=0s)
- [Problema del control de aprendizaje de Cart Pole [Barto, et al. 1983]](http://www.derongliu.org/adp/adp-cdrom/Barto1983.pdf)

Para más ejemplos de aprendizaje por refuerzo en TensorFlow, puede consultar los siguientes recursos:

- [Ejemplos (keras.io) de código de aprendizaje por refuerzo](https://keras.io/examples/rl/)
- [Biblioteca de aprendizaje por refuerzo Agents en TF](https://www.tensorflow.org/agents)
